In [2]:
'''
from google.colab import drive
drive.mount('/content/drive')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [3]:
# !unzip /content/drive/MyDrive/*Utad/3ºAño/IA/Practicas_IA/Practica_2/intervenciones.zip

# Notebook 699: Tarea calificada 2, INAR 23-24

## Generación de texto seq2seq model
## A partir de textos de parlamentarios españoles (anteriores a 2022)

## Nota importante

Esta tarea en su versión 2023-24 surge del excelente trabajo de varios compañeros del curso 2022-23, que aunque yo proporcioné un dataset de textos a partir de las intervenciones de parlamentarios (los líderes de varios partidos en 2021-22, alguno de los cuales ya no está en la política española), hicieron un extraordinario "escrapeo" de la web del Congreso de los Diputados y enriquecieron de forma notable el dataset. Este es el que propongo para esta tarea.

Debo decir que si hay un texto (o lenguaje natural) libre de derechos y especialmente actual, son las intervenciones (estrictamente **públicas**) de los representantes elegidos en elecciones, y que el Congreso debería facilitar, no ya para su uso en estas tareas, sino para cualquier estudioso del español, o de la política, o de la psicología de los políticos.

Por supuesto, esto son opiniones estrictamete mías, en el momento concreto en que las escribo, y sencillamente quiero hacer homenaje a los que colaboraron tanto con este trabajo que espero encontréis interesante.

## ¿De qué trata esta tarea?

Pues ni más ni menos que de generar texto en español a partir de texto de parlamentarios, basado en el tutorial que hemos seguido en clase:

https://www.tensorflow.org/text/tutorials/text_generation?hl=es-419

Para facilitar la tarea se propone un pre-proceso (basado en la tarea 2021-22), y la tarea se concreta en el modelo para generar texto y en las pruebas de la calidad del texto generado.


## Calificación

Está explicada en la entrada correspondiente de Blackboard. Básicamente, hay un mínimo que consiste en proponer tres modelos de red recurrente, uno para cada parlamentario, entrenarlos, y **evaluarlos** generando texto y comentando su calidad.

Para llegar a la máxima nota, propongo poner a dialogar los tres modelos.

Pero por supuesto, valoraré el trabajo de construcción del modelo. Para esta tarea no hay una "medida" como la accuracy en la tarea 1. Será relativamente subjetiva. Por eso parece aconsejable comenzar con modelos pequeños o con pocas etapas e ir refinando.

## Setup

Para facilitar la tarea propongo unas cuantas casillas para cargar en memoria los textos, tres .txt que están incluidos en un .zip.

## Nota importante

La codificación (juego de caracteres) es UTF-8 y creo que debe seguir siendo así. *NO* abráis los .txt con el Notepad de Windows, sino con el Notepad+++ que os permitiría cambiarlo o devolverlo a UTF-8 (o Unicode si queréis).

A pesar que la salida por pantalla (en mi sistema, un Linux) de caracteres ñ y acentuados parece que está mal, luego la generación de texto (insisto, lo he comprobado en mi sistema) es correcta en español.


### Import TensorFlow and other libraries

In [4]:
import tensorflow as tf
import numpy as np
import os
import time
import re
import os


## Lectura de ficheros de datos

In [5]:
datos_abascal   = "intervencionesAbascal.txt"
datos_sanchez   = "intervencionesSanchez.txt"
datos_casado    = "intervencionesCasado.txt"

### Read the data

First, look in the text:

In [6]:
# Read, then decode for py2 compat.
text = open(datos_abascal, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 22573 characters


In [7]:
# Take a look at the first 250 characters in text
print(text[:250])

Señor Sánchez, ¿cómo se atreve usted a hablarme de monólogos si siempre trae las respuestas escritas, si usted nunca contesta a mis preguntas? Conteste por lo menos hoy. ¿Qué va a hacer usted para impedir que VOX siga cruzando las líneas que dice ust


In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

81 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [9]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [10]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [11]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[44, 45, 46, 47, 48, 49, 50], [66, 67, 68]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [12]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [13]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [14]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [15]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


# Fases propuestas para la elaboración del modelo

### 1. Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

Convierte un texto en una secuencia de IDs.

In [16]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(22573,), dtype=int64, numpy=array([37, 48, 75, ..., 62,  6,  1], dtype=int64)>

Crea un conjunto de datos TensorFlow a partir de una secuencia de IDs.

In [17]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

Imprime los primeros 10 caracteres a partir de los IDs en el conjunto de datos.

In [18]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

S
e
ñ
o
r
 
S
á
n
c


Agrupa los IDs en secuencias de longitud 100

In [19]:
seq_length = 100

In [20]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'S' b'e' b'\xc3\xb1' b'o' b'r' b' ' b'S' b'\xc3\xa1' b'n' b'c' b'h' b'e'
 b'z' b',' b' ' b'\xc2\xbf' b'c' b'\xc3\xb3' b'm' b'o' b' ' b's' b'e' b' '
 b'a' b't' b'r' b'e' b'v' b'e' b' ' b'u' b's' b't' b'e' b'd' b' ' b'a'
 b' ' b'h' b'a' b'b' b'l' b'a' b'r' b'm' b'e' b' ' b'd' b'e' b' ' b'm'
 b'o' b'n' b'\xc3\xb3' b'l' b'o' b'g' b'o' b's' b' ' b's' b'i' b' ' b's'
 b'i' b'e' b'm' b'p' b'r' b'e' b' ' b't' b'r' b'a' b'e' b' ' b'l' b'a'
 b's' b' ' b'r' b'e' b's' b'p' b'u' b'e' b's' b't' b'a' b's' b' ' b'e'
 b's' b'c' b'r' b'i' b't' b'a' b's' b','], shape=(101,), dtype=string)


In [21]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Se\xc3\xb1or S\xc3\xa1nchez, \xc2\xbfc\xc3\xb3mo se atreve usted a hablarme de mon\xc3\xb3logos si siempre trae las respuestas escritas,'
b' si usted nunca contesta a mis preguntas? Conteste por lo menos hoy. \xc2\xbfQu\xc3\xa9 va a hacer usted para imped'
b'ir que VOX siga cruzando las l\xc3\xadneas que dice usted que cruzamos? Conteste tambi\xc3\xa9n lo que no me ha con'
b'testado durante toda esta legislatura: \xc2\xbfpor qu\xc3\xa9 minti\xc3\xb3 a los espa\xc3\xb1oles prometi\xc3\xa9ndoles que no pactar\xc3\xada'
b' con estos, con esos y con aquellos?  Cont\xc3\xa9steme a eso y entonces no habr\xc3\xa1 mon\xc3\xb3logos, habr\xc3\xa1 di\xc3\xa1logo p'


Define una función que toma una secuencia y la divide en dos partes: una para la el "input_text" que excluye el último caracter, y otra para el objetivo "target_text" que excluye el primer elemento.

In [22]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [23]:
split_input_target(list("Hola"))

(['H', 'o', 'l'], ['o', 'l', 'a'])

Aplica la función split_input_target a cada secuencia en el conjunto de datos sequences, creando "dataset" que contiene tuplas entrada-objetivo.

In [24]:
dataset = sequences.map(split_input_target)

In [25]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Se\xc3\xb1or S\xc3\xa1nchez, \xc2\xbfc\xc3\xb3mo se atreve usted a hablarme de mon\xc3\xb3logos si siempre trae las respuestas escritas'
Target: b'e\xc3\xb1or S\xc3\xa1nchez, \xc2\xbfc\xc3\xb3mo se atreve usted a hablarme de mon\xc3\xb3logos si siempre trae las respuestas escritas,'


### 2. Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [26]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## 3. Build The Model

Puedes usar cualquiera de los modelos (RNN, LSTM, GRU) que hemos visto en clase. Por supuesto, del tamaño del modelo (capas, neuronas en cada capa) así como de las épocas (más adelante) dependerá el tiempo de proceso en el .fit

In [27]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

MyModel se utiliza para definir un modelo de lenguaje recurrente (RNN) básico en TensorFlow usando Keras. Este tipo de modelo se suele utilizar para tareas relacionadas con el procesamiento de lenguaje natural, en este caso, de generación de texto.

In [28]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

 Crea un modelo con el vocab_size, embedding_dim y las RNN_units especificados anteriormente.


In [29]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 82) # (batch_size, sequence_length, vocab_size)


## 4. Summary y fit del modelo



Aconsejable el uso de GPU en Google Colab, aunque yo he conseguido hacer 200 épocas de mi modelo en un tiempo razonable (menos de 1 hora).

**RECUERDA GRABAR EL MODELO ENTRENADO PARA PODER REUTILIZARLO POSTERIORMENTE**

El modelo consta de tres capas: embedding, gru (unidad de recurrencia), y dense (densa). La capa de embedding tiene 20,992 parámetros, la capa gru tiene 3,938,304 parámetros, y la capa dense tiene 84,050 parámetros. En total, el modelo tiene 4,043,346 parámetros entrenables.

In [31]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  20992     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  84050     
                                                                 
Total params: 4043346 (15.42 MB)
Trainable params: 4043346 (15.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


-Embedding: Asocia a cada palabra un número.

-GRU: Ayuda a entender cómo las palabras se relacionan entre sí a lo largo del texto, como entender el contexto en una conversación.

-Dense: predice la próxima palabra en la secuencia de texto.

Genera índices de palabras muestreadas al azar a partir de las predicciones de un lote de ejemplos

In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [33]:
sampled_indices

array([31, 78, 41, 12, 48, 12, 23, 34, 79, 42, 73, 17, 74, 43, 37, 81, 30,
       27, 38, 80, 60, 12, 23, 64, 24, 79, 47, 57,  5, 22, 78, 58, 16, 21,
       55, 38, 32, 76, 10, 36, 43, 81, 78, 31, 25, 49, 66, 29, 16, 21, 16,
        5, 34, 76, 77, 38,  0, 10, 23, 77, 34, 45, 74, 61, 54, 30, 63, 24,
       42, 53, 49, 51, 42, 12,  5, 42,  3,  5,  9,  2, 50, 18, 53, 27, 32,
       39, 67, 76, 12, 72, 54, 76, 72, 32, 79, 41, 43, 35, 33, 22],
      dtype=int64)

Imprime el texto correspondiente al primer ejemplo en el lote de entrada y las predicciones de la siguiente palabra generadas por el modelo

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ar insultos y odio contra la oposici\xc3\xb3n, lo que no es raro porque tambi\xc3\xa9n han convertido el proceso e'

Next Char Predictions:
 b'M\xe2\x80\x8cX5e5DP\xe2\x80\x94Y\xc3\xa9:\xc3\xadZS\xe2\x80\xa6LHT\xe2\x80\x95q5DuE\xe2\x80\x94dn,C\xe2\x80\x8co9BlTN\xc3\xb33RZ\xe2\x80\xa6\xe2\x80\x8cMFfxJ9B9,P\xc3\xb3\xc3\xbaT[UNK]3D\xc3\xbaPb\xc3\xadrkLtEYjfhY5,Y!,2 g;jHNUy\xc3\xb35\xc3\xa1k\xc3\xb3\xc3\xa1N\xe2\x80\x94XZQOC'


 Función de pérdida

In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

Calcula la pérdida promedio del lote de ejemplos de entrenamiento.

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 82)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4052463, shape=(), dtype=float32)


In [37]:
tf.exp(example_batch_mean_loss).numpy()

81.8793

In [38]:
model.compile(optimizer='adam', loss=loss)

Bloque para borrar el directorio ./training_checkpoints

In [39]:

'''
import shutil
import os

# Directorio que acabamos de crear
directory_to_delete = './training_checkpoints'

# Eliminar el directorio
shutil.rmtree(directory_to_delete)
'''

"\nimport shutil\nimport os\n\n# Directorio que acabamos de crear\ndirectory_to_delete = './training_checkpoints'\n\n# Eliminar el directorio\nshutil.rmtree(directory_to_delete)\n"

Crea un directorio donde se almacenarán los callbacks y se guarda un checkpoint cada 10 epocas

In [40]:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    period=10)

In [41]:
EPOCHS = 400

Se entrena el modelo durante 400 épocas

In [42]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/400
3/3 [==============================] - 6s 1s/step - loss: 4.3519
Epoch 2/400
3/3 [==============================] - 4s 1s/step - loss: 4.5679
Epoch 3/400
3/3 [==============================] - 4s 1s/step - loss: 3.5808
Epoch 4/400
3/3 [==============================] - 4s 1s/step - loss: 3.5481
Epoch 5/400
3/3 [==============================] - 5s 2s/step - loss: 3.4287
Epoch 6/400
3/3 [==============================] - 6s 2s/step - loss: 3.2664
Epoch 7/400
3/3 [==============================] - 5s 2s/step - loss: 3.0814
Epoch 8/400
3/3 [==============================] - 4s 1s/step - loss: 3.0011
Epoch 9/400
3/3 [==============================] - 4s 1s/step - loss: 2.9712
Epoch 10/400
3/3 [==============================] - 6s 2s/step - loss: 2.8841
Epoch 11/400
3/3 [==============================] - 4s 1s/step - loss: 2.8442
Epoch 12/400
3/3 [==============================] - 4s 1s/step - loss: 2.8062
Epoch 13/400
3/3 [==============================] - 4s 1s/step - loss: 2.

Función para ver en que época tiene menos loss para saber mas o menos cual es el mejor modelo


In [43]:
# Acceder a las métricas de entrenamiento
train_loss = history.history['loss']

# Encontrar la época con la pérdida más baja
min_loss_epoch = train_loss.index(min(train_loss)) + 1  # Sumar 1 porque las épocas comienzan desde 1

# Imprimir la época con la pérdida más baja
print(f"Época con la pérdida más baja: {min_loss_epoch}, Loss: {min(train_loss)}")

Época con la pérdida más baja: 400, Loss: 0.02822059392929077


## 5. Genera texto y evalúa su calidad

OneStep se utiliza para definir un modelo de generación de texto de un solo paso basado en un modelo más grande. Genera un único carácter en función de una entrada dada.

In [44]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

Crea un nuevo modelo utilizando OneStep.Este modelo permite generar una sola predicción a la vez, para generar texto de manera iterativa, tomando una palabra predicha como entrada para predecir la siguiente.

In [45]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Utiliza One_step_model para generar texto a partir de la palabra Política. Generará 1000 palabras.

In [46]:
start = time.time()
states = None
next_char = tf.constant(['Política'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Política, añora no se hobre de España y a la vez impidiendo su detención, por instrucciones expresas delirovos mán meresteses de los españoles. Señor Sánchez, usted como ustedes tan an ias que a usted le gustan los aperitivos, pero que la famo a la ganara la oposición de vacuras y con los emportanes sun tormo pera su próximpañía y he e la generanzz. yo tenía unte en par se sus partidos nol trand en el que ha provocado la mayor contracción del producto interior blaní cobrerianaliz y le franómpro interesto el deductor con Parle con el tan o de a lo respira delciónis, la libidad y la tamosiacapie la vivira cuerte que están in la voy a leeran una fació a las rás ilpotratespo que nombre hacia máschedes en usa expEño a las calas―so, se llama ETA.  Señor Sánchez, ha citado usted aquí al señor Azaña, y hemos pensado lo mismo el Rey. Son escabos en Barcelona, que pretendían atentar en nuestro país, habían entrado en patera ilegalmente? Das el barro o lo que parece que ya es tradición socialista

Mas o menos hila palabras pero no no es muy fino. Haría falta, para tener mejores resultados, tener un dataset mas grande, ajustar hiperparámetros ( dimensión del embedding, el número de unidades en la capa GRU, la tasa de aprendizaje) o probar con un modelo mas complejo.


In [47]:
# !zip -r /content/abascal.zip /content/training_checkpoints

## 6. Trabajo adicional

Por ejempl, poner en cadena los tres modelos para que "dialoguen" entre sí

Guardamos el modelo

In [49]:
tf.saved_model.save(one_step_model, 'one_step_abascal')

INFO:tensorflow:Assets written to: one_step_abascal_BORRAR\assets


INFO:tensorflow:Assets written to: one_step_abascal_BORRAR\assets
